In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re

from sklearn.model_selection import train_test_split, GridSearchCV

from sklearn.impute import SimpleImputer

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder

from sklearn.svm import SVR

from sklearn.pipeline import Pipeline, FeatureUnion

from sklearn.metrics import mean_squared_error

import functions as f
import classes as c

In [12]:
train_df = pd.read_csv("datasets/train.csv", na_values=np.NaN).set_index("Id")
test_df = pd.read_csv("datasets/test.csv", na_values=np.NaN).set_index("Id")
corr_matrix = train_df.corr()
# train_df[pd.isnull(train_df)] = np.NaN

train_df.loc[:, "LotAreaSqrt"] = np.sqrt(train_df["LotArea"].values)
train_df.loc[:, "GarageAreaSqrt"] = np.sqrt(train_df["GarageArea"].values)

col_of_interest = ["LotArea", "LotAreaSqrt", "GarageArea", "GarageAreaSqrt", "LotFrontage"]
train_df = train_df[col_of_interest].copy()
train_df.dropna(axis=0, inplace=True)

X = train_df.drop("LotFrontage", axis=1).copy()
y = train_df["LotFrontage"].copy()


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [16]:
tuned_parameters = [{'kernel': ['rbf'], 'gamma': [1e-1, 1e-2, 1e-3, 1e-4],
                     'C': [1, 10, 100, 1000]},
                    {'kernel': ['linear'], 'C': [1, 10, 100, 1000]}]

In [17]:
clf = GridSearchCV(SVR(), tuned_parameters, cv=5, scoring="neg_mean_squared_error")

In [18]:
clf.fit(X_train, y_train)
print("Best parameters set found on development set:")
print(clf.best_params_)

Best parameters set found on development set:
{'C': 10, 'gamma': 0.0001, 'kernel': 'rbf'}


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


In [ ]:
train_df = train_df[["LotArea", "GarageArea", "GarageCars", "LotFrontage"]].copy()
train_df.loc[:, "LotAreaSqrt"] = np.sqrt(train_df["LotArea"].values)
train_df.loc[:, "GarageAreaSqrt"] = np.sqrt(train_df["GarageArea"].values)
train_df.dropna(axis=0, inplace=True)
X = train_df.drop("LotFrontage", axis=1).copy()
y = train_df["LotFrontage"].copy()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [ ]:
model.fit(X_train, y_train)

In [ ]:
y_train_pr = model.predict(X_train)
y_test_pr = model.predict(X_test)
mse_train = mean_squared_error(y_train, y_train_pr)
mse_test = mean_squared_error(y_test, y_test_pr)